In [1]:
# Author: Matthew Green
# Date: 12/7/2017
# License: MIT

import numpy as np
import pandas as pd

### Introduction

I downloaded a CSV of [City of Chicago employee salary data](https://data.cityofchicago.org/Administration-Finance/Current-Employee-Names-Salaries-and-Position-Title/xzkq-xp2w), 
which includes the names, titles, departments and salaries of Chicago employees. I was
interested to see whether men and women earn similar salaries for similar roles.

City data don't report the gender of employees, so I used an employee's first name
as a proxy, which is explained in more detail below.

In [2]:
df = pd.read_csv('Current_Employee_Names__Salaries__and_Position_Titles.csv')

Here's what the first 10 records look like:

In [3]:
df.head(10)

,Name,Job Titles,Department,Full or Part-Time,Salary or Hourly,Typical Hours,Annual Salary,Hourly Rate
0,"AARON, JEFFERY M",SERGEANT,POLICE,F,Salary,NaN,$101442.00,NaN
1,"AARON, KARINA",POLICE OFFICER (ASSIGNED AS DETECTIVE),POLICE,F,Salary,NaN,$94122.00,NaN
2,"AARON, KIMBERLEI R",CHIEF CONTRACT EXPEDITER,GENERAL SERVICES,F,Salary,NaN,$101592.00,NaN
3,"ABAD JR, VICENTE M",CIVIL ENGINEER IV,WATER MGMNT,F,Salary,NaN,$110064.00,NaN
4,"ABARCA, EMMANUEL",CONCRETE LABORER,TRANSPORTN,F,Hourly,40.0,NaN,$36.18
5,"ABASCAL, REECE E",TRAFFIC CONTROL AIDE-HOURLY,OEMC,P,Hourly,20.0,NaN,$19.86
6,"ABBASI, CHRISTOPHER",STAFF ASST TO THE ALDERMAN,CITY COUNCIL,F,Salary,NaN,$50436.00,NaN
7,"ABBATACOLA, ROBERT J",ELECTRICAL MECHANIC,AVIATION,F,Hourly,40.0,NaN,$46.10
8,"ABBATE, JOSEPH L",POOL MOTOR TRUCK DRIVER,STREETS & SAN,F,Hourly,40.0,NaN,$35.60
9,"ABBATEMARCO, JAMES J",FIRE ENGINEER-EMT,FIRE,F,Salary,NaN,$103350.00,NaN


To make grouping and matching first names easier, I extracted the first name from each 
employee record and lower-cased it:

In [4]:
df['Name'] = df['Name'].apply(lambda s: s.split(',')[1].split()[0].lower())

To simplify the analysis, I restricted my attention to full-time employees with a salary.

In [5]:
workers = df[(df['Salary or Hourly']=='Salary') & (df['Full or Part-Time']=='F')]

### Gender prediction

To estimate the gender of an employee based on his or her first name, I used a data set of 
[baby names](https://www.ssa.gov/oact/babynames/limits.html). For each unique name, I counted
how many times, from years 1940 to 2016, that name had been given to a boy versus a girl. If 
the name was more frequently given to boys, then I predicted the gender associated with the
name to be male, and vice-versa for female.

In [6]:
# Data are in seperate CSV files per year, and are accumulated here
name_data = []
for yob in range(1940, 2017):
    name_data.append(pd.read_csv('names/yob' + str(yob) + '.txt', 
                     header=0, names=['Name', 'Gender', 'Count']))
names = pd.concat(name_data, axis=0)

In [7]:
# Lower-case first name so that it can be joined with the workers dataframe
names['Name'] = names['Name'].str.lower() # So

In [8]:
names.head(10)

,Name,Gender,Count
0,barbara,F,36732
1,patricia,F,32662
2,judith,F,22380
3,betty,F,22075
4,carol,F,21760
5,nancy,F,19732
6,linda,F,18368
7,shirley,F,17876
8,sandra,F,16151
9,margaret,F,15058


In [9]:
# Count how often a name is given to boys and girls
gender_frequency = names.groupby(['Name', 'Gender']).sum().reset_index()

In [10]:
def predict_gender(df):
    max_idx = df['Count'].idxmax()
    return df.loc[max_idx]

In [11]:
# Select the more frequent gender for each name
gender_guess = gender_frequency.groupby('Name').agg(predict_gender).reset_index()

In [12]:
gender_guess.sample(10)

,Name,Gender,Count
42256,kahmal,M,6
60853,mui,F,5
18003,cristhofer,M,56
19156,damonique,F,215
72902,sanyi,F,57
79609,tahesha,F,165
8607,aujanae,F,188
89194,yorvin,M,5
2220,aiyla,F,170
64331,nwamaka,F,5


The above list of names and associated genders can be combined with the worker data to 
predict the gender of each Chicago employee:

In [13]:
workers = pd.merge(workers, gender_guess, on='Name', how='inner')

In [14]:
workers.head(10)

,Name,Job Titles,Department,Full or Part-Time,Salary or Hourly,Typical Hours,Annual Salary,Hourly Rate,Gender,Count
0,jeffery,SERGEANT,POLICE,F,Salary,NaN,$101442.00,NaN,M,228050
1,jeffery,POLICE OFFICER,POLICE,F,Salary,NaN,$93354.00,NaN,M,228050
2,jeffery,POLICE OFFICER,POLICE,F,Salary,NaN,$90024.00,NaN,M,228050
3,jeffery,POLICE OFFICER,POLICE,F,Salary,NaN,$96060.00,NaN,M,228050
4,jeffery,POLICE OFFICER,POLICE,F,Salary,NaN,$84054.00,NaN,M,228050
5,jeffery,BRIDGE OPERATOR,TRANSPORTN,F,Salary,NaN,$66900.00,NaN,M,228050
6,jeffery,POLICE OFFICER,POLICE,F,Salary,NaN,$93354.00,NaN,M,228050
7,jeffery,POLICE OFFICER,POLICE,F,Salary,NaN,$93354.00,NaN,M,228050
8,jeffery,FIELD ANALYST,PROCUREMENT,F,Salary,NaN,$52176.00,NaN,M,228050
9,jeffery,POLICE OFFICER,POLICE,F,Salary,NaN,$87006.00,NaN,M,228050


### Analysis

I wanted to know wether men and women were paid equally if they shared the same job title and
department. To answer this, I specifically looked at full-time, salaried employees, and jobs
for which both men and women were employed under the same title and department.

For example, given the job title POLICE OFFICER in the POLICE department, a position for which
both men and women are employed, do male and female officers have similar salaries? More
generally, are men and women paid equally across all job titles and departments?

In [15]:
# Focus on these columns
workers = workers[['Job Titles', 'Department', 'Gender', 'Annual Salary']]

In [16]:
# Convert salaries from strings to floats
workers['Annual Salary'] = workers['Annual Salary'].apply(lambda s: float(s.strip('$')))

In [17]:
worker_groups = workers.groupby(['Job Titles', 'Department'])

In [18]:
# Remove jobs for which only men or only women are employed
worker_groups = worker_groups.filter(lambda df: len(df) > 1)

In [19]:
worker_groups = worker_groups.groupby(['Job Titles', 'Department', 'Gender'])

In [20]:
group_avgs = worker_groups.mean().reset_index(level='Gender')

In [21]:
group_avgs.head(10)

Gender  Annual Salary
Job Titles            Department                             
A/SUPRV REDISTRICTING BOARD OF ELECTION      F        44604.0
                      BOARD OF ELECTION      M        49236.0
ACCIDENT ADJUSTER     GENERAL SERVICES       F        53340.0
                      GENERAL SERVICES       M        78612.0
ACCOUNTANT I          FINANCE                F        63282.0
                      FINANCE                M        62872.0
                      PUBLIC LIBRARY         F        76548.0
                      PUBLIC LIBRARY         M        76548.0
ACCOUNTANT II         FAMILY & SUPPORT       M        84516.0
                      FINANCE                F        77418.0